In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Instructions
rubric={points}

You will earn points for following these instructions and successfully submitting your work on Gradescope.  

### Group wotk instructions

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2.
  
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).
- If you would like to use late tokens for the homework, all group members must have the necessary late tokens available. Please note that the late tokens will be counted for all members of the group.   


### General submission instructions

- Please **read carefully
[Use of Generative AI policy](https://ubc-cs.github.io/cpsc330-2025W1/syllabus.html#use-of-generative-ai-in-the-course)** before starting the homework assignment. 
- **Run all cells before submitting:** Go to `Kernel -> Restart Kernel and Clear All Outputs`, then select `Run -> Run All Cells`. This ensures your notebook runs cleanly from start to finish without errors.
  
- **Submit your files on Gradescope.**  
   - Upload only your `.ipynb` file **with outputs displayed** and any required output files.
     
   - Do **not** submit other files from your repository.  
   - If you need help, see the [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/).  
- **Check that outputs render properly.**  
   - Make sure all plots and outputs appear in your submission.
     
   - If your `.ipynb` file is too large and doesn't render on Gradescope, also upload a PDF or HTML version so the TAs can view your work.  
- **Keep execution order clean.**  
   - Execution numbers must start at "1" and increase in order.
     
   - Notebooks without visible outputs may not be graded.  
   - Out-of-order or missing execution numbers may result in mark deductions.  
- **Follow course submission guidelines:** Review the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html) for detailed guidance on completing and submitting assignments. 
   
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
words = ["chocolate", "hockey", "europe", "car"]

In [7]:
for word in words:
    print(f"\nTop similar words to '{word}':")
    print(glove_wiki_vectors.most_similar(word, topn = 5))


Top similar words to 'chocolate':
[('cream', 0.848685622215271), ('cake', 0.7893113493919373), ('caramel', 0.784212589263916), ('candy', 0.7699085474014282), ('cheese', 0.7688507437705994)]

Top similar words to 'hockey':
[('basketball', 0.8041675090789795), ('football', 0.7833694219589233), ('nhl', 0.7604308724403381), ('soccer', 0.7440758347511292), ('baseball', 0.7312085628509521)]

Top similar words to 'europe':
[('european', 0.8116024732589722), ('america', 0.7819679975509644), ('asia', 0.7772932648658752), ('countries', 0.7699106335639954), ('world', 0.7643163204193115)]

Top similar words to 'car':
[('vehicle', 0.8630837798118591), ('truck', 0.8597879409790039), ('cars', 0.837166965007782), ('driver', 0.8185910582542419), ('driving', 0.781263530254364)]


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [8]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [9]:
for pair in word_pairs: 
    word1 = pair[0]
    word2 = pair[1]
    similarity = glove_wiki_vectors.similarity(word1, word2)
    print(f"Similarity for ({word1}, {word2}): {similarity:.4f}\n")

Similarity for (coast, shore): 0.7000

Similarity for (clothes, closet): 0.5463

Similarity for (old, new): 0.6432

Similarity for (smart, intelligent): 0.7553

Similarity for (dog, cat): 0.8798

Similarity for (tree, lawyer): 0.0767



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [10]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [11]:
for word in test_words:
    in_vocab = word in glove_wiki_vectors
    print(f"{word}: {in_vocab}")

covididiot: False
fomo: False
frenemies: True
anthropause: False
photobomb: False
selfie: False
pxg: False
pacg: False
cct: True
escc: True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [12]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [13]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [14]:
glove_wiki_vectors.similarity("aboriginal", "success")

np.float32(0.14283238)

In [15]:
glove_wiki_vectors.similarity("white", "success")

np.float32(0.351824)

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [16]:
analogy("christian", "bad", "muslim")

christian : bad :: muslim : ?


,Analogy word,Score
0,worse,0.651489
1,worst,0.634681
2,because,0.615937
3,situation,0.605221
4,hurt,0.602466
5,scared,0.597197
6,dangerous,0.591691
7,too,0.590602
8,worried,0.587227
9,fear,0.587100


In [17]:
analogy("muslim", "bad", "christian")

muslim : bad :: christian : ?


,Analogy word,Score
0,good,0.645781
1,thing,0.597388
2,lot,0.591887
3,really,0.589379
4,things,0.583396
5,big,0.574034
6,stuff,0.572241
7,n't,0.571348
8,little,0.569936
9,too,0.566916


In [18]:
glove_wiki_vectors.similarity("christian", "terrorism")

np.float32(0.26753694)

In [19]:
glove_wiki_vectors.similarity("muslim", "terrorism")

np.float32(0.48266083)

In [20]:
glove_wiki_vectors.similarity("man", "intelligent")

np.float32(0.4394357)

In [21]:
glove_wiki_vectors.similarity("woman", "intelligent")

np.float32(0.3758921)

In [22]:
glove_wiki_vectors.similarity("man", "dumb")

np.float32(0.37205106)

In [23]:
glove_wiki_vectors.similarity("woman", "dumb")

np.float32(0.24443853)

In [24]:
glove_wiki_vectors.similarity("white", "violent")

np.float32(0.27509207)

In [25]:
glove_wiki_vectors.similarity("black", "violent")

np.float32(0.35746297)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

I tested a number of potential biases surrounding race, religion, and gender in the GloVe embeddings database. Although Wikipedia is a relatively moderated source, the model still encodes a number of potentially harmful social stereotypes. For example, the analogy "christian : bad :: muslim : ?" ranks words such as a "dangerous", "fear", and "hurt" among the top completions, while reversing the analogy produces "good" as the top result. Similarly, the similarity between "muslim" and "terrorism" was substantially higher than the similarity between "christian" and "terrorism" (0.48 vs 0.27). This suggests that the embedding space implicitly associated Muslim identity with violence, indicating that the model has absorbed some Islamophobic biases from its training corpus.

I also found it quite strange that word "man" was deemed more closely related to both the words "intelligent" and "dumb" than the word "woman" was to either of these adjectives. Although subtle, these asymmetries reflect stereotypical associations between gender and intellectual ability, with the algorithm associating both poles of intelligence as more "manly". Overall I found this exercise really informative, and I'll be sure to consider these implicit biases as I continue working with these embeddings tools. 


<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [26]:
from sklearn.datasets import fetch_20newsgroups

In [27]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [28]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [29]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [30]:
def preprocess(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.is_stop:
            continue
        if token.is_punct:
            continue
        if token.like_num or token.like_url or token.like_email:
            continue
        if token.is_alpha and token.pos_ in ["NOUN", "PROPN", "VERB", "ADJ"]:
            tokens.append(token.lemma_.lower())
    return " ".join(tokens)

In [31]:
df["text_pp"] = df["text"].apply(preprocess)

In [32]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 4


I experimented with a few groupings, however I ended up settling for n_components = 8 since the dataset includes 8 categories from the newsgroups data. Increasing beyond this resulted in some topics seeming a bit too repetitive, and decreasing below this resulted in some topics seeming too broad, so I think 8 is a good happy-medium for our objectives.

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    max_df = 0.9,
    min_df = 5, 
    stop_words = "english"
)

X = vectorizer.fit_transform(df["text_pp"])

In [34]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(
    n_components = 8, 
    random_state = 42
)

lda_model.fit(X)

,n_components,8
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 5

Topic #0: Programming & Software
Top words hint at coding, software usage, servers, and files.

Topic #1: Sports & Games
Focus on teams, players, seasons, winning, and gameplay.

Topic #2: Religion & Philosophy
Mentions of Jesus, God, church, atheism, belief, and arguments.

Topic #3: Hockey & NHL
Specifically about hockey teams, NHL, games, and leagues.

Topic #4: Middle Eastern Politics & Conflicts
References to Israel, Armenia, Turkey, governments, wars, and rights.

Topic #5: General Religious/Belief Systems
Words like God, believe, question, think, people — similar to Topic #2 but perhaps broader/general discussion.

Topic #6: Guns, Crime, & Law
Focus on firearms, weapons, crime rates, control, and laws.

Topic #7: General Conversation / Social Interactions
Very generic terms: say, people, tell, know, come

In [35]:
words = vectorizer.get_feature_names_out()

In [36]:
for i in range(lda_model.n_components):
    topic = lda_model.components_[i]
    sorted_indices = topic.argsort()[::-1]
    top_words = [words[j] for j in sorted_indices[:10]]
    print("Top words for topic #{}:".format(i))
    print(top_words)
    print()

Top words for topic #0:
['file', 'program', 'use', 'image', 'window', 'available', 'include', 'run', 'server', 'work']

Top words for topic #1:
['game', 'team', 'play', 'year', 'good', 'player', 'think', 'win', 'season', 'time']

Top words for topic #2:
['jesus', 'god', 'people', 'argument', 'church', 'atheist', 'know', 'thing', 'believe', 'think']

Top words for topic #3:
['hockey', 'la', 'team', 'pt', 'new', 'game', 'nhl', 'san', 'league', 'det']

Top words for topic #4:
['people', 'israel', 'armenian', 'turkish', 'government', 'right', 'state', 'israeli', 'armenians', 'war']

Top words for topic #5:
['god', 'think', 'people', 'know', 'believe', 'good', 'question', 'time', 'say', 'thing']

Top words for topic #6:
['gun', 'firearm', 'weapon', 'jews', 'crime', 'control', 'university', 'law', 'handgun', 'rate']

Top words for topic #7:
['say', 'people', 'know', 'come', 'tell', 'time', 'woman', 'think', 'happen', 'start']



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [37]:
doc_topic_dist = lda_model.transform(X)

for i in range(5):
    topic_assignment = doc_topic_dist[i].argmax()
    print(f"Document {i} is assigned to topic {topic_assignment}")

Document 0 is assigned to topic 4
Document 1 is assigned to topic 0
Document 2 is assigned to topic 5
Document 3 is assigned to topic 0
Document 4 is assigned to topic 4


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-based filtering works by suggesting items to a user based on the characteristics/features of items that they have interacted with previously. For instance, if I have watched a number of history-based documentaries, then a recommender system using this sort of filtering will recommend other documentaries that share similar attributes (ie. genre, actors, plot keywords, etc...), under the assumption that because I have watched a lot of these shows/documentaries, that I must like this sort of show/documentary.

2. One downside to recommender systems is that they can sometimes be repetitive in what they suggest by reinforcing a user's existing preferences. As a result, a user might miss out on new items that may interest them, because the system is primarily focused on things that it knows the user has historically liked. This can bore users over time, which is why companies engaging in this practice need to find the correct balance between recommending items that the user has liked as well as well as suggesting some new items every now and then as well. Another consequence of recommender systems is that they can reinforce certain biases which exist in the data used to train them. For example, if the system is trained on historical user interactions that reflect societal or cultural biases (similarly to what we analyzed in exercise 1.4), it may preferentially recommend content that aligns with those biases, unintentionally amplifying stereotypes or limiting exposure to diverse perspectives.

3. Transfer learning is the process of taking a pretrained language model and then tuning this model slightly to fit your specific task/goals. This means the model often already understands general language, such as how words and sentences work, and you would just need to teach the model some task-specific patterns/recognitions which helps to save time. This is quite similar to what we did in Q2, where NLP already has an understanding of how the language works.

<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top. 

Here is a quick checklist before submitting: 

- [ ] Restart kernel, clear outputs, and run all cells from top to bottom.  
- [ ] `.ipynb` file runs without errors and contains all outputs.  
- [ ] Only `.ipynb` and required output files are uploaded (no extra files).  
- [ ] Execution numbers start at **1** and are in order.  
- [ ] If `.ipynb` is too large and doesn't render on Gradescope, also upload a PDF/HTML version.  
- [ ] Reviewed the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html).  

![](img/eva-well-done.png)